In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('./current_offers.xlsx')
print(data.shape)
data.head()

(384, 6)


,NAME,WEB,CASH_BACK_HEIGHT,TRANCHE_STMT_COUNT,OFFER_TYPE,ADVERT_TEXT
0,ANEX Tour,https://www.anextour.com/,0,6.0,SPECIAL_CREDIT,На покупку по кредитной карте не будут начисля...
1,Гостиница Альфа,http://www.alfa-hotel.ru/,15,NaN,STANDARD,Гостиница Альфа 4* – разумный выбор для отдыха...
2,MirCli,https://mircli.ru/?utm_source=Tinkoff&utm_medi...,0,4.0,SPECIAL_CREDIT,На покупку по кредитной карте не будут начисля...
3,MIXIT,https://mixit.ru,10,NaN,STANDARD,MIXIT - лаборатория современной косметики. Это...
4,Tinkoff.Travel совместно с Booking.com,https://travel.tinkoff.ru/hotels/?utm_source=p...,10,NaN,STANDARD,Бронирование отелей через Тинькофф Путешествия...


In [3]:
data = data[~data['WEB'].isna()]
data.shape

(370, 6)

In [4]:
data = data[~((data['CASH_BACK_HEIGHT'] == 0) & (data['OFFER_TYPE'] != 'SPECIAL_CREDIT'))]
data.shape

(357, 6)

In [5]:
def unique(data, cols):
    new_data = []
    for gr in data.groupby('NAME'):    
        gr = gr[1]
        gr = gr.drop_duplicates(subset=cols)

        if len(gr) > 1:
            if len(gr['OFFER_TYPE'].unique()) == 1:
                gr = gr[gr['CASH_BACK_HEIGHT'] == gr['CASH_BACK_HEIGHT'].max()]
            else:
                gr1 = gr[gr['CASH_BACK_HEIGHT'] == gr['CASH_BACK_HEIGHT'].max()]
                gr2 = gr[gr['OFFER_TYPE'] == 'SPECIAL_CREDIT']
                gr = pd.concat([gr1, gr2])

        assert len(gr) <= 2

        new_data.append(gr)

    return pd.concat(new_data).sort_index()

In [6]:
data = unique(data, cols=['CASH_BACK_HEIGHT'])
data.shape

(350, 6)

In [7]:
web = pd.read_excel('./offers_with_categories.xlsx')
print(web.shape)
web.head()

(688, 7)


,OFFER_ID,NAME,CASH_BACK_HEIGHT,TRANCHE_STMT_COUNT,OFFER_TYPE,ADVERT_TEXT,CATEGORY_NAME
0,5352,Tinkoff.Travel совместно с Rentalcars,0,NaN,STANDARD,Аренда автомобилей через Тинькофф Путешествия ...,Каникулы в Сочи
1,5352,Tinkoff.Travel совместно с Rentalcars,0,NaN,STANDARD,Аренда автомобилей через Тинькофф Путешествия ...,Путешествия
2,5354,Tinkoff.Travel совместно с Booking.com,10,NaN,STANDARD,Бронирование отелей через Тинькофф Путешествия...,Каникулы в Сочи
3,5354,Tinkoff.Travel совместно с Booking.com,10,NaN,STANDARD,Бронирование отелей через Тинькофф Путешествия...,Путешествия
4,5515,Gett,10,NaN,STANDARD,Gett — один из мировых лидеров заказа такси. С...,Для детей


In [8]:
web = web[(web['CATEGORY_NAME'] == 'Спорт') | (web['CATEGORY_NAME'] == 'Еда и продукты')]
web.shape

(72, 7)

In [9]:
web = web[~web['CATEGORY_NAME'].isna()]
web.shape

(72, 7)

In [10]:
web = web[~((web['CASH_BACK_HEIGHT'] == 0) & (web['OFFER_TYPE'] != 'SPECIAL_CREDIT'))]
web.shape

(63, 7)

In [11]:
web = unique(web, cols=['CASH_BACK_HEIGHT', 'CATEGORY_NAME'])
web.shape

(59, 7)

In [12]:
def merge(cat):
    offer_with_cats = []
    for _, row in data.iterrows():
        cats = web[web['NAME'] == row['NAME']]
        cats = cats[cats['CATEGORY_NAME'] == cat]
        if len(cats) == 0:
            continue

        assert len(cats) == 1 or (len(cats) == 2 and len(cats['OFFER_TYPE'].unique()) == 2), cats
        offer_with_cats.append(row)
        
    return pd.DataFrame(offer_with_cats)

In [13]:
data.shape, web.shape

((350, 6), (59, 7))

In [14]:
sport = merge('Спорт')
sport.shape

(29, 6)

In [15]:
sport.to_excel('sport.xls', index=False)

In [16]:
eda = merge('Еда и продукты')
len(eda)

29

In [17]:
eda.to_excel('food.xls', index=False)

In [18]:
names = np.intersect1d(sport['NAME'], eda['NAME'])
d = []
for name in names:
    d.append(sport[sport['NAME'] == name])
pd.concat(d).to_excel('intersect.xls', index=False)